In [ ]:
import numpy as np
import pandas as pd
from math import pi
from bokeh.models.glyphs import Circle, Patches, Wedge
from bokeh.plotting import figure
from bokeh.models import Range1d, PanTool, ResetTool, HoverTool, ColumnDataSource, LabelSet
from bokeh.io import output_notebook, output_file, show
from utils import parseSite, draw_pitch

In [ ]:
italy = parseSite(66) # Italy
wales = parseSite(144) # Wales
length = 105
width = 68

In [ ]:
df = pd.read_parquet('D:/socceranalytics/italywales.pq')
position_cols = [col for col in df if "_x" in col or "_y" in col or "_type" in col]
x_pos = [col for col in df if "_x" in col]
y_pos = [col for col in df if "_y" in col]
type_cols = [col for col in df if "_type" in col]
id_cols = [col for col in df if "_id" in col]
df[x_pos] = df[x_pos].astype(float) / 100.0
df[y_pos] = df[y_pos].astype(float) / 100.0

In [ ]:
player_map = dict()
for player in italy:
    player_map[player.id] = player.name
for player in wales:
    player_map[player.id] = player.name

In [ ]:
random_point = df.sample(1)
new_index = random_point[x_pos].T.index.to_list()
new_index = [index.strip("_x") for index in new_index]
positions = random_point[x_pos].T
positions.index = new_index
positions.columns = ["x"]
positions["y"] = random_point[y_pos].T.to_numpy()
positions["type"] = random_point[type_cols].T.to_numpy()
positions["id"] = random_point[id_cols].T.to_numpy()
ball_pos = positions.iloc[0]
positions = positions.iloc[1:]
positions["name"] = positions["id"].map(player_map)
positions['x'] = length / 2.0 + positions['x']
positions['y'] = width / 2.0 + positions['y']
positions['type'] = positions['type'].apply(lambda x: -1 if x is None else int(x))
home = positions[positions['type'] == 0]
away = positions[positions['type'] == 1]

In [ ]:
output_notebook()

In [ ]:
TOOLTIPS = [
    ("name", "@name"),
    ("(x,y)", "($x, $y)"),
]
Italy = ColumnDataSource(home)
Wales = ColumnDataSource(away)
p = draw_pitch(tooltips = TOOLTIPS)
p.circle(x="x", y="y", size=8, fill_alpha=0.5, color="red",source=Italy)
p.circle(x="x", y="y", size=8, fill_alpha=0.5, color="yellow",source=Wales)
output_file("interactive_pitch.html") #save the file
show(p)